In [ ]:
 # mount google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
pip install pillow-heif Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 25.2 MB/s eta 0:00:00


HEIC to PNG

In [ ]:

import os
from PIL import Image
import pillow_heif
import shutil

# Register HEIF opener
pillow_heif.register_heif_opener()

input_folder = '/content/drive/MyDrive/cataract/Normal/Normal'
output_folder = '/content/drive/MyDrive/cataract/Normal_converted'

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Check if folder exists and list contents
if not os.path.exists(input_folder):
    print(f"❌ Input folder not found: {input_folder}")
else:
    files = os.listdir(input_folder)
    print(f"📁 Found {len(files)} files in the folder.")

    for filename in files:
        input_path = os.path.join(input_folder, filename)

        # Convert HEIC to PNG
        if filename.lower().endswith(".heic"):
            png_filename = os.path.splitext(filename)[0] + ".png"
            png_path = os.path.join(output_folder, png_filename)

            try:
                image = Image.open(input_path)
                image.save(png_path, "PNG")
                print(f"✅ Converted: {filename} -> {png_filename}")
            except Exception as e:
                print(f"❌ Failed to convert {filename}: {e}")

        # Copy existing PNG files
        elif filename.lower().endswith(".png"):
            output_path = os.path.join(output_folder, filename)
            try:
                shutil.copy2(input_path, output_path)
                print(f"📥 Copied: {filename}")
            except Exception as e:
                print(f"❌ Failed to copy {filename}: {e}")


📁 Found 558 files in the folder.
✅ Converted: IMG_0337 (1).HEIC -> IMG_0337 (1).png
✅ Converted: IMG_0379 (1).HEIC -> IMG_0379 (1).png
✅ Converted: IMG_0370.HEIC -> IMG_0370.png
✅ Converted: IMG_0377.HEIC -> IMG_0377.png
✅ Converted: IMG_0544 (1).HEIC -> IMG_0544 (1).png
✅ Converted: IMG_0535 (1).HEIC -> IMG_0535 (1).png
✅ Converted: IMG_0332 (1).HEIC -> IMG_0332 (1).png
✅ Converted: IMG_1120.HEIC -> IMG_1120.png
✅ Converted: IMG_0338 (1).HEIC -> IMG_0338 (1).png
✅ Converted: IMG_0546 (1).HEIC -> IMG_0546 (1).png
✅ Converted: IMG_0330 (1).HEIC -> IMG_0330 (1).png
✅ Converted: IMG_0317.HEIC -> IMG_0317.png
✅ Converted: IMG_0316.HEIC -> IMG_0316.png
✅ Converted: IMG_0380 (1).HEIC -> IMG_0380 (1).png
✅ Converted: IMG_0542 (1).HEIC -> IMG_0542 (1).png
✅ Converted: IMG_0507.HEIC -> IMG_0507.png
✅ Converted: IMG_0532 (1).HEIC -> IMG_0532 (1).png
✅ Converted: IMG_0502.HEIC -> IMG_0502.png
✅ Converted: IMG_0371.HEIC -> IMG_0371.png
✅ Converted: IMG_0403 (1).HEIC -> IMG_0403 (1).png
✅ Converted

In [ ]:
import cv2
import numpy as np
import os

def crop_pupil(image_path, save_dir, filename='cropped_pupil.jpg'):
    # Create output directory
    os.makedirs(save_dir, exist_ok=True)

    # Read image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_blurred = cv2.medianBlur(gray, 5)

    # Detect circles using HoughCircles (pupil usually appears as a dark circle)
    circles = cv2.HoughCircles(
        gray_blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
        param1=100, param2=20, minRadius=10, maxRadius=80
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :1]:  # Only take first circle
            x, y, r = circle
            padding = 10
            x1, y1 = max(x - r - padding, 0), max(y - r - padding, 0)
            x2, y2 = min(x + r + padding, img.shape[1]), min(y + r + padding, img.shape[0])
            cropped = img[y1:y2, x1:x2]

            save_path = os.path.join(save_dir, filename)
            cv2.imwrite(save_path, cropped)
            print(f"✅ Pupil cropped and saved to: {save_path}")
            return

    print("⚠️ No pupil detected.")

# Example usage:
image_path = '/content/IMG_0139.jpg'  # Your uploaded image path
save_dir = 'pupil_dataset'
crop_pupil(image_path, save_dir)


✅ Pupil cropped and saved to: pupil_dataset/cropped_pupil.jpg


In [ ]:
import os
import json
import cv2

# Paths
coco_json_path = "/content/drive/MyDrive/cataract/cocojson/eyelash/labels_my-project-name_2025-07-13-05-12-43.json"  # Path to your COCO JSON file
images_dir = "/content/drive/MyDrive/cataract/Normal_converted"               # Folder containing the original images
output_dir = "/content/drive/MyDrive/cataract/Normal_pupil"      # Folder to save cropped regions
os.makedirs(output_dir, exist_ok=True)

# Load COCO annotations
with open(coco_json_path, "r") as f:
    coco = json.load(f)

# Build image id to file name mapping
image_id_map = {img["id"]: img["file_name"] for img in coco["images"]}

# Start cropping
for ann in coco["annotations"]:
    image_id = ann["image_id"]
    image_filename = image_id_map[image_id]
    image_path = os.path.join(images_dir, image_filename)
    img = cv2.imread(image_path)

    if img is None:
        print(f"Image not found: {image_path}")
        continue

    # Get bounding box: [x, y, width, height]
    x, y, w, h = map(int, ann["bbox"])
    cropped = img[y:y+h, x:x+w]

    # Save cropped region
    crop_filename = f"{image_id}_{ann['id']}.jpg"
    cv2.imwrite(os.path.join(output_dir, crop_filename), cropped)

print("✅ Cropping complete. Check folder:", output_dir)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/cataract/cocojson/eyelash/labels_my-project-name_2025-07-13-05-12-43.json'

In [ ]:
import os

# Define the folder path
folder_path = '/content/drive/MyDrive/cataract/Normal/Normal'

# Define valid image extensions
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif', '.heic')

# Count image files
image_count = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(image_extensions))

print(f"Total number of images: {image_count}")

import os

# Define the folder path
folder_path = '/content/drive/MyDrive/cataract/Normal_converted'

# Define valid image extensions
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif', '.heic')

# Count image files
image_count = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(image_extensions))

print(f"Total number of images: {image_count}")

# Define the folder path
folder_path = '/content/drive/MyDrive/cataract/dataset/Normal_pupil'

# Define valid image extensions
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif', '.heic')

# Count image files
image_count = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(image_extensions))

print(f"Normal images: {image_count}")


# Define the folder path
folder_path = '/content/drive/MyDrive/cataract/dataset/Mild_pupil'

# Define valid image extensions
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif', '.heic')

# Count image files
image_count = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(image_extensions))

print(f"Mild images: {image_count}")

# Define the folder path
folder_path = '/content/drive/MyDrive/cataract/dataset/Severe_pupil'

# Define valid image extensions
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif', '.heic')

# Count image files
image_count = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(image_extensions))

print(f"Severe images: {image_count}")






Total number of images: 558
Total number of images: 557
Normal images: 549
Mild images: 427
Severe images: 436


In [ ]:
import os
import json
import cv2

# Paths
coco_json_path = "/content/drive/MyDrive/cataract/cocojson/eyelash/labels_my-project-name_2025-07-13-05-12-43.json"
images_dir =  "/content/drive/MyDrive/cataract/eyelash_dataset"
output_dir = "/content/drive/MyDrive/cataract/eyelash_dataset_v2"
os.makedirs(output_dir, exist_ok=True)

# Load COCO annotations
with open(coco_json_path, "r") as f:
    coco = json.load(f)

# Mapping from image ID to file name
id_to_filename = {img["id"]: img["file_name"] for img in coco["images"]}
filename_to_id = {v: k for k, v in id_to_filename.items()}

# Group annotations by image_id
from collections import defaultdict
annotations_by_id = defaultdict(list)
for ann in coco["annotations"]:
    annotations_by_id[ann["image_id"]].append(ann)

# Load Haar cascade (fallback)
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Supported extensions
image_extensions = ('.jpg', '.jpeg', '.png')

# Start processing
for img_name in os.listdir(images_dir):
    if not img_name.lower().endswith(image_extensions):
        continue

    img_path = os.path.join(images_dir, img_name)
    img = cv2.imread(img_path)
    if img is None:
        print("Could not read:", img_path)
        continue

    image_id = filename_to_id.get(img_name)

    if image_id in annotations_by_id:
        # Crop using COCO bbox
        for ann in annotations_by_id[image_id]:
            x, y, w, h = map(int, ann["bbox"])
            cropped = img[y:y+h, x:x+w]
            out_name = f"{os.path.splitext(img_name)[0]}_coco_{ann['id']}.jpg"
            cv2.imwrite(os.path.join(output_dir, out_name), cropped)
    else:
        # Fallback: Haar cascade eye detection
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
        if len(eyes) > 0:
            for i, (x, y, w, h) in enumerate(eyes[:1]):  # First detection
                cropped = img[y:y+h, x:x+w]
                out_name = f"{os.path.splitext(img_name)[0]}_haar.jpg"
                cv2.imwrite(os.path.join(output_dir, out_name), cropped)
        else:
            # Optional: fallback center crop (e.g., 100x100 in center)
            h_img, w_img = img.shape[:2]
            cx, cy = w_img // 2, h_img // 2
            size = 100
            cropped = img[cy-size//2:cy+size//2, cx-size//2:cx+size//2]
            out_name = f"{os.path.splitext(img_name)[0]}_center.jpg"
            cv2.imwrite(os.path.join(output_dir, out_name), cropped)

print("✅ Done: All 439 images processed with COCO, Haar or center fallback.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/cataract/cocojson/eyelash/labels_my-project-name_2025-07-13-05-12-43.json'

In [ ]:
# 2. Install required library
!pip install pillow-heif


In [ ]:
# 3. Imports
import os
from PIL import Image
import pillow_heif

# Register HEIF opener
pillow_heif.register_heif_opener()


In [ ]:
# 4. Set paths
input_folder = '/content/drive/MyDrive/cataract/Mild/Mild'
output_folder = '/content/drive/MyDrive/cataract/Mild_converted'
os.makedirs(output_folder, exist_ok=True)

# 5. Convert HEIC to PNG and log failures
failed_files = []

if not os.path.exists(input_folder):
    print(f"❌ Input folder not found: {input_folder}")
else:
    files = os.listdir(input_folder)
    heic_files = [f for f in files if f.lower().endswith('.heic')]
    print(f"📁 Found {len(heic_files)} HEIC files.")

    for filename in heic_files:
        heic_path = os.path.join(input_folder, filename)
        png_filename = os.path.splitext(filename)[0] + ".png"
        png_path = os.path.join(output_folder, png_filename)

        try:
            image = Image.open(heic_path)
            image.save(png_path, "PNG")
            print(f"✅ Converted: {filename}")
        except Exception as e:
            print(f"❌ Failed: {filename} — {e}")
            failed_files.append((filename, str(e)))

# 6. Save failed conversions
failed_path = os.path.join(output_folder, "failed_conversions.txt")
with open(failed_path, "w") as f:
    for file, error in failed_files:
        f.write(f"{file} : {error}\n")

print(f"\n✅ Converted: {len(heic_files) - len(failed_files)}")
print(f"❌ Failed: {len(failed_files)}")
print(f"📝 Log saved to: {failed_path}")


📁 Found 311 HEIC files.
✅ Converted: IMG_1124.HEIC
✅ Converted: IMG_1145.HEIC
✅ Converted: IMG_0474 (1).HEIC
✅ Converted: IMG_0324 (1).HEIC
✅ Converted: IMG_0301 (1).HEIC
✅ Converted: IMG_0302 (1).HEIC
✅ Converted: IMG_0571.HEIC
✅ Converted: IMG_0325.HEIC
✅ Converted: IMG_0470.HEIC
✅ Converted: IMG_0475 (1).HEIC
✅ Converted: IMG_0576 (1).HEIC


KeyboardInterrupt: 

In [ ]:
# 7. Count PNGs to verify
png_files = [f for f in os.listdir(output_folder) if f.lower().endswith('.png')]
print(f"🖼️ PNG files ready: {len(png_files)}")


🖼️ PNG files ready: 311


In [ ]:
# 8. Zip the folder
!zip -r /content/mild_converted.zip "/content/drive/MyDrive/cataract/Mild_converted"


  adding: content/drive/MyDrive/cataract/Mild_converted/ (stored 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_1124.png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_1145.png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_0474 (1).png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_0324 (1).png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_0301 (1).png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_0302 (1).png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_0571.png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_0325.png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_0470.png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_0475 (1).png (deflated 0%)
  adding: content/drive/MyDrive/cataract/Mild_converted/IMG_0576 (1).png (deflate

In [ ]:
# 9. Download the ZIP
from google.colab import files
files.download('/content/mild_converted.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Remove Eyelash from an Image**

In [ ]:
import os
import cv2
import numpy as np

# Paths
input_folder = "/content/drive/MyDrive/cataract/dataset/Normal_pupil"     # <-- change this
output_folder = "/content/drive/MyDrive/cataract/dataset/Normal_pupil_filter"   # <-- change this
os.makedirs(output_folder, exist_ok=True)

# Loop through images
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Failed to load {filename}")
            continue

        h, w = img.shape[:2]
        mask = np.zeros((h, w), dtype=np.uint8)

        # Circle centered in the image, adjust radius if needed
        center = (w // 2, h // 2)
        radius = min(h, w) // 3  # or use //2 for larger circle

        # Draw white circle on black background
        cv2.circle(mask, center, radius, 255, thickness=-1)

        # Apply circular mask
        masked_img = cv2.bitwise_and(img, img, mask=mask)

        # Optional: convert background to white instead of black
        background_white = np.ones_like(img, dtype=np.uint8) * 255
        final_img = np.where(mask[..., np.newaxis] == 255, masked_img, background_white)

        # Save the result
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, final_img)

print("✅ All images processed with circular crop.")


✅ All images processed with circular crop.


**Remove Noises from the images**

In [ ]:
import os
import cv2
import numpy as np

input_folder = "/content/drive/MyDrive/cataract/dataset/Normal_pupil_filter"     # <-- set this
output_folder = "/content/drive/MyDrive/cataract/dataset/Normal_pupil_filter_img"   # <-- set this
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Skipping {filename}")
            continue

        h, w = img.shape[:2]
        center = (w // 2, h // 2)
        radius = min(h, w) // 3

        # Create circular mask
        mask = np.zeros((h, w), dtype=np.uint8)
        cv2.circle(mask, center, radius, 255, thickness=-1)
        masked_img = cv2.bitwise_and(img, img, mask=mask)

        # Convert to LAB and apply CLAHE to L-channel
        lab = cv2.cvtColor(masked_img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        lab = cv2.merge((cl, a, b))
        enhanced_img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

        # Apply median blur to remove salt & pepper noise
        blurred = cv2.medianBlur(enhanced_img, 3)

        # Optional: Apply bilateral filter to keep edges sharp
        denoised = cv2.bilateralFilter(blurred, d=9, sigmaColor=75, sigmaSpace=75)

        # Make background white
        background_white = np.ones_like(img, dtype=np.uint8) * 255
        final = np.where(mask[..., np.newaxis] == 255, denoised, background_white)

        # Save
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, final)

print("✅ All images processed with circular crop + noise removal.")


✅ All images processed with circular crop + noise removal.


**Create a dataset by using yolov8 model**

In [ ]:
from ultralytics import YOLO
import cv2
import os

# === PATH SETUP ===
model_path = "/content/drive/MyDrive/cataract/yolo_output/yolov8_pupil5/weights/best.pt"   # Your trained pupil detection model
input_folder = "/content/drive/MyDrive/cataract/Normal_converted"       # Folder with original images
output_folder = "/content/drive/MyDrive/cataract/dataset/Normal_pupil"    # Folder to save cropped pupils
os.makedirs(output_folder, exist_ok=True)

# === LOAD MODEL ===
model = YOLO(model_path)

# === INFERENCE AND CROP ===
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Failed to load {filename}")
            continue

        results = model(img)
        boxes = results[0].boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)
            cropped = img[y1:y2, x1:x2]

            # Optional: Resize crop to standard size
            cropped = cv2.resize(cropped, (256, 256))

            out_name = f"{os.path.splitext(filename)[0]}_crop{i+1}.jpg"
            out_path = os.path.join(output_folder, out_name)
            cv2.imwrite(out_path, cropped)

print("✅ All pupil crops saved.")


ModuleNotFoundError: No module named 'ultralytics'